(alglhco)=
# Algoritmos LHCO 2020
El rendimiento de los distintos métodos que utilizaron los participantes de las LHCO 2020 sólo se puede comparar mediante los resultados que obtuvieron. Sin embargo, como se puede observar en la {numref}`lhco-resultados`, la mayoría de los modelos no reportaron el error asociado al resultado obtenido. Más aún, cada participante reportó los resultados y rendimiento del modelo de acuerdo al método utilizado, dificultando la comparación directa de los análisis. 

Uno de los objetivos principales de este trabajo es comparar directamente algunos modelos participantes de las LHCO 2020. Para ello, es necesario poder reproducir el resultado de dichos modelos. Sin embargo, estos deben cumplir múltiples de los requisitos para hacer investigación reproducible explicados en la {numref}`rpd-investigacion`, a manera de asegurar que se pueda obtener el resultado de los modelos en un rango de tiempo adecuado para el desarrollo de este trabajo. 

A continuación, se hablará del análisis de los algoritmos participantes a nivel de la reproducibilidad de sus resultados y se explicarán los modelos a comparar en este trabajo.

(alglhco-rep)=
## Reproducibilidad
Para escoger los algoritmos a comparar, se hizo una revisión extensiva de la información proporcionada por los participantes mencionados en la {numref}`lhc-participantes`. Como se explica en la {numref}`rpd-investigacion`, para poder reproducir resultados en este contexto es necesario principalmente que se encuentre pública la información sobre el pre-procesamiento de los datos, el código del modelo, instrucciones para utilizarlo, información acerca del entorno computacional y licencia. De cada participante se encontró lo siguiente,

```{table} Información disponible de cada participante de las LHCO 2020
:name: alglhc-participantesrep

| Nombre | Pre-procesamiento | Código| Instrucciones | Entorno| Licencia|
|:------:|:-----------------:|:-----:|:-------------:|:------:|:-------:|
| VRNN   |$\checkmark$ |$\checkmark$ |$\checkmark$ |- |- |
| ANODE  |- |- |- |- |- |
| BuHuLaSpa|- |$\checkmark$ |- |- |- |
| GAN-AE |$\checkmark$ |$\checkmark$ |$\checkmark$ |$\checkmark$ |- |
| GIS    |- |- |- |- |- |
|  LDA   |$\checkmark$ |$\checkmark$ |$\checkmark$ |$\checkmark$ |- |
|  PGAE  |$\checkmark$ |$\checkmark$ |$\checkmark$ |$\checkmark$ |- |
| Reg. Likelihoods|- |- |- |- |- |
|UCluster|$\checkmark$ |$\checkmark$ |$\checkmark$ |$\checkmark$ |$\checkmark$ |
| CWoLa  |- |$\checkmark$ |- |- |- |
|CWoLa AE Compare|- |- | |- |- |
|Tag N' Train|$\checkmark$ |$\checkmark$ |- |$\checkmark$ |- |
| SALAD  |- |$\checkmark$ |- |$\checkmark$ |- |
|SA-CWoLa|- |$\checkmark$ |- |$\checkmark$ |- |
|Deep Ensemble|- |$\checkmark$ |$\checkmark$ |$\checkmark$ |- |
|Factorized Topics|- |$\checkmark$ |- |- |$\checkmark$ |
| QUAK   |- |- |- |- |- |
| LSTM   | - |- |- |- |- |- |
```
Un resumen de general de la información proporcionada por todos los algoritmos se encuentra en la {numref}`alglhco-repfigure`

In [1]:
from myst_nb import glue
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15})

plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels

datos = {'Pre-procesamiento':6, 'Código':12, 'Instrucciones':6, 'Entorno':8, 'Licencia':2}
items = list(datos.keys())
valores = list(datos.values())
colores = ['darkorange', 'crimson', 'green', 'blue', 'purple']

fig = plt.figure(figsize = (9, 6), facecolor='white')
 
# creating the bar plot
plt.bar(items, valores, color = colores,width = 0.6)
plt.xlabel("Información disponible")
plt.ylabel("Nro. de participantes")
plt.title("Reproducibilidad de los algoritmos de las LHCO 2020")
plt.ylim([0,18])

plt.savefig('./../../figuras/alglhco-repfig.png', bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
plt.close()

```{figure} ./../../figuras/alglhco-repfig.png
---
name: alglhco-repfigure
width: 60%
---
Resumen de la reproducibilidad de los participantes de las LHCO 2020.
```
En la {numref}`alglhco-repfigure` se observa que la mayoría de los algoritmos participantes no poseen suficiente información pública para poder reproducir sus resultados, lo que inmmediatamente límita las opciones de modelos a utilizar en este trabajo. De los 18 participantes, sólo *UCluster* cumple con todas las características. *GAN-AE*, *LDA*  y *PGAE* cumplen con cuatro de las características. Para este trabajo se escogieron ***UCluster*** y ***GAN-AE*** debido a la simplicidad de la información proporcionada. Los algoritmos se explicarán siguiendo {cite}`Kasieczka_2021`.

(alglhco-ucluster)=
## UCluster
UCluster (Unsupervised Clustering){cite}`Mikuni_2021` es un método que agrupa puntos con características similares, inmersos en una representación reducida de los datos de eventos. Esta representación se crea utilizando una red neuronal para reducir la dimensionalidad de los datos. El método es independiente de la arquitectura de la red neuronal. 

Para crear el encjae se hace una *clasificación de masa de jet por partícula*, es decir, a cada partícula perteneciente a un jet se le asigna una etiqueta proporcional a la masa del jet al que pertenece. Estas etiquetas se crean definiendo 20 intervalos equidistantes entre 10 y 1000 GeV. 

En particular, el modelo debe aprender la masa del jet al que la partícula pertenece y cuáles partículas pertenecen al musmo jet. El modelo utiliza las primeras 100 partículas asociadas a los dos jets más masivos de cada evento y consta de dos tareas, una de clasificación y otra de agrupamiento. La función de pérdida conjunta es:

$$
    L = L_{focal} + \beta L_{cluster}.
$$ (loss-ucluster)

Un valor de $\beta=10$ se utiliza para darle a las componentes el mismo orden de magnitud relativo.

$L_{focal}$ hace referencia a la función de pérdida focal, que es comúnmente utilizada para clasificar datos con etiquetas altamente desbalanceadas{cite}`Kasieczka_2021`.

$$
    L_{focal}=-\frac{1}{N}\sum_j^N\sum_m^M y_{j,m}(1-p_{\theta,m}(x_j))^\gamma \log(p_{\theta,m}(x_j))
$$ (focal-loss)

donde $p_{\theta,m}(x_j)$ es la confianza de la red para el evento $x_j$, con parámetros entrenables $\theta$, de ser clasificado como clase $m$. $y_{j,m}$ es 1 si la clase $m$ es correctamente asignada al evento $x_j$ y 0 de otra forma. Utilizaron $\gamma=2$.

La función de pérdida del agrupamiento fue definida como,

$$
    L_{cluster}=\frac{1}{N}\sum_k^K\sum_j^N\|f_\theta(x_j)-\mu_k\|^2\pi_{jk}.
$$ (cluster-loss)

La distancia entre el evento $x_j$ y el centroide $\mu_k$ se calcula en el espacio de encaje $f_\theta$, creado por la clasificación. $\pi_{jk}$ pesa la importancia de cada evento para el objetivo de agrupación y está definido como:

$$
    \pi_{jk}=\frac{e^{-\alpha\|f_\theta(x_j)-\mu_k\|}}{\sum_{k'}e^{-\alpha\|f_\theta(x_j)-\mu_k\|}}
$$ (peso-clusterloss)

con el hiperparámetro $\alpha$.

El valor inicial de los centroides se hace utilizando el algoritmo K-means, para luego realizar el entrenamiento utilizando la función de pérdida conjunta en la ecuación {eq}`loss-ucluster`. 

Como se mencionó anteriormente, el modelo es libre de la arquitectura de la red neuronal. Sin embargo, la implementación utilizada en las LHCO 2020 utiliza ABCNet, una red basada en grafos donde cada partícula reconstruida es un nodo del grafo. La arquitectura y la capa del encaje se encuentran en el siguiente diagrama,

```{figure} ../../figuras/bench-ABCNet.png
---
width: 500px
name: bench-ABCNet
---
Diagrama de la red ABCNet utilizada por UCluster.
```
Para crear las GAPLayers se utiliza la distancia entre los 10 vecinos mas cercanos de cada partícula. La primera GAPLayer utiliza la distancia $\Delta R=\sqrt{\Delta\eta^2+\Delta\phi^2}$ y la segunda utiliza las distancias euclídeas.

(alglhco-ganae)=
## GAN-AE
El método GAN-AE intenta asociar un discriminante y un codificador automatico como una red generativa antagónica siguiendo los pasos a continuación:

La **red discriminativa** es un perceptrón multicapas (MLP). Inicialmente es entrenado utilizando la entropia cruzada binaria (ec.{eq}`binary-crossentropy`) en una muestra mezclada de eventos reconstruidos y originales, con el objetivo de exponer las débilidades del codificador automatico.

El **codificador automatico** es entrenado utilizando una función de pérdida que combina la reconstrucción del error (en este caso, la distancia euclídea media (MED) entre la entrada y la salida) y la entropía binaria cruzada.

$$  
    \mathcal{L}_{AE} = \mathcal{L}_{BC}+\epsilon\times\text{MED}+\alpha\times\text{DisCo}
$$ (loss-ae)

donde $\epsilon$ y $\alpha$ son hiperparámetros para balancear los pesos de cada término y $\text{DisCo}$ es la correlación de distancia, para descorrelacionar el error de reconstrucción de la masa invariante. En este caso, $\mathcal{L}_{BC}$ se evalúa dando eventos reconstruidos a la red discriminativa, pero esta vez con la etiqueta "equivocada", con el objetivo de engañarla.

Por último, se evalúa el codificador automatico utilizando una *figura de mérito*:

$$
    \text{FoM}=\text{MED}+(1-\text{Media salida MLP}).
$$ (fom)

El término $1-\text{Media salida MLP}$ es cercano a cero a medida que el codificador automático engaña mejor al MLP.

Estos tres pasos se repiten en un búcle y una vez que el codificador automatico ha sido entrenado se descarta la red discriminativa. Una vez descartada, el codificador automatico utiliza las distancias euclídeas como característica discriminativa.

Los participantes utilizaron el método anteriormente descrito en conjunto con *BumpHunter*, un algoritmo que compara la distribución de los datos con datos de referencia y evalúa el valor p y la significancia de cualquier desviación. Sin embargo, la implementación en este trabajo se limita al uso del algoritmo GAN-AE, que es la parte publicada de este método.
